# Dependencies

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Install Libraries

!pip install roboflow
!pip install git+https://github.com/ultralytics/ultralytics.git@main

In [ ]:
#Import libraries

from IPython.display import display, Image


import ultralytics
ultralytics.checks()
from ultralytics import YOLO


from roboflow import Roboflow

# Collect Landslide Dataset

In [ ]:
rf = Roboflow(api_key="F1mQjaZRGePHUiH3RoqO")
project = rf.workspace("idk-mukck").project("durian-r6ykr")
dataset = project.version(1).download("yolov8")


In [ ]:
dataset.location

In [ ]:
!ls {dataset.location}

# Prepare Dataset

In [ ]:
file_path = f"{dataset.location}/data.yaml"
yaml_content = f"""
names:
- '101'
- BlackThorn
- D13
- D2
- D24
- D88
- GoldenPheonix
- KH
- KucingTidur
- Musangking
nc: 10
roboflow:
  license: CC BY 4.0
  project: durian-r6ykr
  url: https://universe.roboflow.com/idk-mukck/durian-r6ykr/dataset/1
  version: 1
  workspace: idk-mukck
test: {dataset.location}/test/images
train: {dataset.location}/train/images
val: {dataset.location}/valid/images
"""
print(yaml_content)
try:
    with open(file_path, 'w') as file:
        file.write(yaml_content)
    print(f"YAML content has been overwritten in '{file_path}'")
except FileNotFoundError:
    print(f"File '{file_path}' not found.")

# Train The Model

In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data={dataset.location}/data.yaml epochs=20 imgsz=640 plots=True

In [ ]:
import glob
from PIL import Image
from IPython.display import display

def display_images_in_folder(folder_path):
    # Use glob to get a list of all image files in the folder
    image_files = glob.glob(f"{folder_path}/*.jpg") + glob.glob(f"{folder_path}/*.png")

    # Loop through the image files and display them
    for image_file in image_files:
        img = Image.open(image_file)
        display(img)
        
# display_images_in_folder("/kaggle/working/runs/detect/train")

# Test The Model

In [ ]:
!yolo task=detect mode=predict model=/kaggle/working/runs/detect/train/weights/best.pt conf=.36 source="https://cms-image-bucket-production-ap-northeast-1-a7d2.s3.ap-northeast-1.amazonaws.com/images/7/8/8/0/41930887-3-eng-GB/Cropped-1661370688photo_SXM2022081400006127.jpg" save=True


In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'/kaggle/working/runs/detect/predict7/*')[0:90]:
      display(Image(filename=image_path, width=600))
      print("\n")

# Download Model

- #### The model can be downloaded simply by clicking on the download button next to the file name.

- #### The file name can be found under this folder /kaggle/working/runs/segment/train/weights/best.pt


- #### Do note the folder name might differ if you trained the model muluiple times. Each time you train the model, a one is added to "train", so if you train it 3 times then the folder name will be "train3"

In [ ]:
!zip -r durian_detector_1.zip /kaggle/working/runs/detect/train